# Intermediate Neural Network in Keras

Bulding an Intermediate Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [2]:
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Load data

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#### Preprocess data
Flatten and normalise input data.

In [4]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')

In [5]:
X_train /= 255
X_test /= 255

In [6]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### ReLUs

In [7]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

class ReLUs(Activation):
    
    def __init__(self, activation, **kwargs):
        super(ReLUs, self).__init__(activation, **kwargs)
        self.__name__ = 'relus'

def relus(Z):
    e_param = 0.07
    pi = K.variable((3.14))
    m = e_param * (K.sigmoid(K.sin(Z)) - K.sigmoid(K.cos(Z)) * K.exp(K.sqrt(pi)))
    A = K.maximum(m, Z)
    return A

get_custom_objects().update({'relus': ReLUs(relus)})

#### Design Neural Network architecture

In [8]:
model = Sequential()
model.add(Dense(128, activation = 'relu', kernel_initializer='glorot_uniform', input_shape = (784,)))
model.add(Dropout(0.2))
model.add(Dense(256, activation = 'relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.3))
model.add(Dense(1024, activation = 'relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              263168    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                10250     
Total para

#### Callbacks

In [10]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-deepnet-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=20)


if not os.path.exists('model_output'):
    os.makedirs('model_output')
    
tensorboard = TensorBoard("../logs/deepnet-mnist-relus")

#### Configure model

In [11]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [12]:
model.fit(X_train, y_train, batch_size = 128, epochs = 50, verbose = 1,
          validation_split = 0.1, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 5s 92us/step - loss: 0.3996 - acc: 0.8749 - val_loss: 0.1161 - val_acc: 0.9660
Epoch 2/50
54000/54000 [==============================] - 5s 92us/step - loss: 0.1789 - acc: 0.9458 - val_loss: 0.0979 - val_acc: 0.9682
Epoch 3/50
54000/54000 [==============================] - 5s 87us/step - loss: 0.1368 - acc: 0.9582 - val_loss: 0.0858 - val_acc: 0.9727
Epoch 4/50
54000/54000 [==============================] - 6s 107us/step - loss: 0.1164 - acc: 0.9636 - val_loss: 0.0806 - val_acc: 0.9770
Epoch 5/50
54000/54000 [==============================] - 5s 88us/step - loss: 0.0998 - acc: 0.9695 - val_loss: 0.0834 - val_acc: 0.9748
Epoch 6/50
54000/54000 [==============================] - 5s 95us/step - loss: 0.0913 - acc: 0.9722 - val_loss: 0.0693 - val_acc: 0.9800
Epoch 7/50
54000/54000 [==============================] - 5s 93us/step - loss: 0.0847 - acc: 0.9735 - val_loss: 0.0635 - val_acc

#### Test Predictions

In [13]:
saved_model = keras.models.load_model('model_output/weights-deepnet-mnist.hdf5')
predictions = saved_model.predict_classes(X_test, verbose = 2)
print(predictions)

[7 2 1 ..., 4 5 6]


#### Test Final Accuracy

In [15]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

10000/10000 [==============================] - 1s 98us/step
Final loss: 0.0678, final accuracy: 0.9826
